In [1]:
import glob
import torch
import numpy as np
import torch.nn as nn
import scipy.io as sio
import matplotlib.pyplot as plt
import torch.nn.functional as F
import scipy.ndimage as ndimage
from torchvision import transforms, utils
from toolz.curried import pipe, curry, compose
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt

import csv
import numpy as np


from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import os

import scores

In [3]:
import argparse
import datetime
import numpy as np
import math
import os
import gc

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader



## Reading in the Data

In [ ]:
path_f=os.getcwd()

path_f_1=os.path.join(path_f, 'data')


names=[]
for files_txts in os.listdir(path_f_1):
    if files_txts.endswith(".csv"):
        #print(files_txts)
        names.append(files_txts)

print(names)
path_train=os.path.join(path_f_1, names[0])
path_test=os.path.join(path_f_1, names[1])

df_train=pd.read_csv(path_train)
print(df_train.shape)
print(path_train)

## Feature Engineering

In [ ]:
stab_vector=df_train['stabilityVec'].values
y=[]
for x in stab_vector:
    #print(x)
    a=np.fromstring(x[1:-1],sep=',').astype(int)
    y.append(a)
y=np.array(y) 

df_tmp = pd.DataFrame(y, columns = ['A', 'A91B', 'A82B','A73B','A64B','A55B','A46B','A37B','A28B','A19B','B'])
stab_vec_list=[ 'A91B', 'A82B','A73B','A64B','A55B','A46B','A37B','A28B','A19B','Stable_compunds']

df_train=df_train.drop("stabilityVec",axis=1) #removing the results which originally are a string
feature_cols=list(df_train)



df_train['formulaA']=df_train['formulaA_elements_Number']
df_train['formulaB']=df_train['formulaB_elements_Number']


df_train=pd.concat([df_train, df_tmp],axis=1)

y_all=df_train[stab_vec_list[0:-1]]


df_tmp_stable = pd.DataFrame( columns = ['Stable_compunds'])
df_tmp_stable['Stable_compunds']=np.logical_not(y_all.sum(axis=1)==0).astype(int) ## A one means it has a stable value  a 0 

df_train=pd.concat([df_train, df_tmp_stable],axis=1)

y_all=df_train[stab_vec_list]
print(df_train.shape)


print(df_train.shape)


df_stable=df_train.loc[np.logical_not(y_all.sum(axis=1)==0)]
print(df_stable.shape)
df_train.head()

In [ ]:
#X_train_new=df_stable[feature_cols] #training only on stable elements
#y_target=df_stable[stab_vec_list]
###

X_train_new=df_train[feature_cols]   #training  on all elements
y_target=df_train[stab_vec_list]

print(X_train_new.shape,y_target.shape)

In [ ]:
# Normalizing such that the magnitude is one
from sklearn.preprocessing import normalize

X_train_new_mag_1=normalize(X_train_new, axis=1) # vector magnitude is one
X_train_new_mag_1=pd.DataFrame(data=X_train_new_mag_1,columns=feature_cols)
print(X_train_new_mag_1.shape)


## Normalizing by Zscore
from scipy.stats import zscore
X_train_new_Z_score=X_train_new.apply(zscore)
print(X_train_new_Z_score.shape)



## Normalizing so that range is 0-1
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_train_new_0_1=min_max_scaler.fit_transform(X_train_new)
X_train_new_0_1=pd.DataFrame(data=X_train_new_0_1,columns=feature_cols)
print(X_train_new_0_1.shape)


## Normalizing so that range is -1 to 1
from sklearn import preprocessing
max_abs_scaler = preprocessing.MaxAbsScaler()
X_train_new_m1_p1=max_abs_scaler.fit_transform(X_train_new)
X_train_new_m1_p1=pd.DataFrame(data=X_train_new_m1_p1,columns=feature_cols)
print(X_train_new_m1_p1.shape)


# Using PCA as input
X_train_4_PCA=df_train[feature_cols]
indx_4_PC=X_train_4_PCA.index
X_train_new_mag_1_PCA=normalize(X_train_4_PCA, axis=1)


pca = PCA()
pca.fit(X_train_new_mag_1_PCA)
components = pca.components_[:20,:]
new_data = np.dot(X_train_new_mag_1_PCA, components.T)
X_train_new_PCA=new_data

print(X_train_new_PCA.shape)


## Taking Logarithm of High Values

X_train_new_log=X_train_new.copy()
X_train_new_log[X_train_new_log>100]=X_train_new_log[X_train_new_log>100].apply(np.log)
print(X_train_new_log.shape)

In [ ]:
device = torch.device("cpu")#device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(98, 60),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(60, 20),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(20, 10),
            nn.ReLU())

    def forward(self, batch):
        return self.layers(batch)
    

In [ ]:
model = Network().to(device)

In [ ]:
class feature_Dataset(Dataset):
    """Face landmarks Dataset"""
    
    def __init__(self, X_features, y_target):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """        
        self.X_features=X_features
        self.y_target=y_target
        assert len(X_features) == len(y_target)
        


        self.X_features = torch.from_numpy(X_features.values).float()
        self.y_target = torch.from_numpy(y_target.values).float()
        
    def __len__(self):
        return len(self.X_features)

    def __getitem__(self, idx):
        
        
        item = {'index': idx}
        item['features'] = self.X_features[idx,:]
        item['outputs'] = self.y_target[idx,:]
        return item


In [ ]:
## test-train split
X_train, X_test, y_train, y_test = train_test_split(X_train_new_m1_p1, y_target,
                                                    test_size=.15,
                                                    shuffle=True,
                                                    random_state=42)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)
#X_train.head()

In [ ]:
X_train.head()

In [ ]:
batch_size = 64
train_dataset = feature_Dataset(X_train, y_train) ## Now you can feed any X-train and Y-train. normalize it and feed the highest pearson correlated one

trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=6)

val_dataset = feature_Dataset(X_test, y_test)

valloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=6)

In [ ]:
item = train_dataset[0]

x = item['features'][None]
print("features: ", x)
y = item['outputs']
print("outputs: ", y)


In [ ]:
y.shape

# Training

In [ ]:
def mse_loss(y1, y2):
    """standard MSE definition"""
    return ((y1 - y2) ** 2).sum() / y1.data.nelement() * 500


def class_loss(y1, y2):
    """Standard MAE definition"""
    return ((y1 - y2).abs()).sum() 

In [ ]:
# Loss and optimizer
num_epochs = 20
learning_rate = .00001
#criterion = nn.CrossEntropyLoss()
criterion = class_loss
#criterion = nn.MultiLabelSoftMarginLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total_step = len(trainloader)
print("No. of steps per epoch: %d" % total_step)

In [ ]:
print(criterion)
print(optimizer)
for epoch in range(10):
    if epoch > 0:
        learning_rate = .00001
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif epoch >= 10:
        learning_rate = 0.0002
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    elif epoch > 80:
        learning_rate = 0.0001
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for i, item in enumerate(trainloader):
        model.train()
        x = item['features'].to(device)
        target = item['outputs'].to(device)
        # Forward pass
        output = model(x)
        output1=Variable(data=(output>.9).float(), requires_grad=True)
        loss = criterion(output1, target)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i % 35 == 0:
            niter = 5
            val_loss=0
            for i1, item1 in enumerate(valloader):
                x_val = item1['features'].to(device)
                y_val = item1['outputs'].to(device)
                model.eval()
                y_pre = model(x_val)
                y_pre1=Variable(data=(y_pre>.9).float(), requires_grad=True)
                val_loss += criterion(y_pre1, y_val)

                if i1 == niter-1:
                    break
                if val_loss.item()/niter < 6.5:
                    torch.save(model.state_dict(), "learnt_weights_C0_%d_%d" % (i, epoch))
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.6f}, Val Loss: {:.6f}' 
                   .format(epoch+1, num_epochs, i, total_step, loss.item(), val_loss.item()/niter))
        
        

In [ ]:
y_pre1[:5]

In [ ]:
y_val[:5]

In [ ]:
output1[:1]

In [ ]:
target[:1]